In [1]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
from omegaconf import OmegaConf
import numpy as np
from data import FeaturesTrainDataset, FeaturesDatasetOOD

In [2]:
from train import FastflowTrainer

cfg_path = "/workspaces/ood/fastflow/config.yaml"
cfg = OmegaConf.load(cfg_path)

In [3]:
train_dataset = FeaturesTrainDataset(
    "/workspaces/ood/data/feature_maps/cifar10train_wide_resnet50_layer2_layer3_layer4_1000_img_per_class.npy",
    ["layer2", "layer3", "layer4"],
    [(10000, 512, 28, 28), (10000, 1024, 14, 14), (10000, 2048, 7, 7)],
)

In [4]:
test_dataset = FeaturesDatasetOOD(
    "/workspaces/ood/data/feature_maps/cifar10test_wide_resnet50_layer2_layer3_layer4_100_img_per_class.npy",
    "/workspaces/ood/data/feature_maps/svhntest_wide_resnet50_layer2_layer3_layer4_100_img_per_class.npy",
    ["layer2", "layer3", "layer4"],
    [(1000, 512, 28, 28), (1000, 1024, 14, 14), (1000, 2048, 7, 7)],
)

In [5]:
train_dataloader = DataLoader(
    train_dataset, batch_size=cfg.trainer.batch_size, shuffle=True
)
test_dataloader = DataLoader(
    test_dataset, batch_size=cfg.trainer.batch_size, shuffle=False
)

In [8]:
from train import FastflowTrainer

trainer = FastflowTrainer(cfg)

trainer.train(train_dataloader, test_dataloader)

CNF coder: 512
CNF coder: 1024
CNF coder: 2048


  0%|          | 0/625 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py:64: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:174.)
  return default_collate([torch.as_tensor(b) for b in batch])
 28%|██▊       | 177/625 [02:14<05:39,  1.32it/s]


KeyboardInterrupt: 